<h1 align=center><font size = 5>CAPSTONE PROJECT FINAL WEEK</font></h1>

<h2 align=center><font size = 5>Choose a location to open new business in Hanoi, Vietnam</font></h2>

### 1. Import necessary libraries

In [1]:
# install geocoder to get coordinates later
!pip install geocoder

     |████████████████████████████████| 102kB 9.3MB/s eta 0:00:01


In [2]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0           conda-forge
    geopy:          

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         713 KB

The following NEW packages will be INSTALLED:

    altair:  4.1.0-py_1 conda-forge
    branca:  0.4.1-py_0 conda-forge
    folium:  0.5.0-py_0 conda-forge
    vincent: 0.4.4-py_1 conda-forge


altair-4.1.0         | 614 KB    | ##################################### | 100% 
folium-0.5.0    

### 2. Download and explore dataset

<p>Hanoi has a total of 30 neighborhoods (districts). In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 30 neighborhoods as well as the the latitude and logitude coordinates of each neighborhood.</p>
<p>In this project, the list of neighborhoods shall be imported from wikipedia: https://en.wikipedia.org/wiki/Category:Districts_of_Hanoi, using pandas</p>

In [3]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Districts_of_Hanoi").text

In [4]:
import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

In [5]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [6]:
# create a list to store neighborhood data
neighborhoodList = []

In [7]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [8]:
# create a new DataFrame from the list
df = pd.DataFrame({"Neighborhood": neighborhoodList})
df.head()

,Neighborhood
0,Ba Đình District
1,Ba Vì District
2,Bắc Từ Liêm District
3,Cầu Giấy District
4,Chương Mỹ District


In [9]:
# remove unncessary word
df['Neighborhood'] = df['Neighborhood'].str.replace('District','')
df.head()

,Neighborhood
0,Ba Đình
1,Ba Vì
2,Bắc Từ Liêm
3,Cầu Giấy
4,Chương Mỹ


In [10]:
# print the number of rows of the dataframe
df.shape

(30, 1)

### 3. Get the latitude & longtitude of boroughs

In [11]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

In [12]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Hanoi, Vietnam'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [13]:
# call the function to get the coordinates, store in a new list using list comprehension
coordinates = [ get_latlng(neighborhood) for neighborhood in df["Neighborhood"].tolist() ]

In [14]:
coordinates

[[21.022010000000023, 105.81934000000007],
 [21.083330000000046, 105.38333000000006],
 [21.062170000000037, 105.76941000000005],
 [21.035610795172605, 105.80597332891945],
 [21.039549981678203, 105.84793000000008],
 [21.08979000000005, 105.66402000000005],
 [21.155300000000068, 105.84872000000007],
 [20.68605516155898, 105.7368790189703],
 [21.01644340273899, 105.92092252294405],
 [20.97031000000004, 105.78181000000006],
 [21.155050000000074, 105.73429000000004],
 [21.08627000000007, 105.77028000000007],
 [21.07906999942315, 105.81928008050087],
 [21.028579952247846, 105.83206997446484],
 [21.04207268806938, 105.86516239134127],
 [21.159590000000037, 105.73480000000006],
 [21.014550076605694, 105.85160997446485],
 [21.00813000000005, 105.76650000000006],
 [21.233560000000068, 105.43031000000008],
 [21.03227000000004, 105.85244000000006],
 [21.00334251230042, 105.65226736541494],
 [21.275149235529767, 105.8890703072445],
 [21.032795923976675, 105.83013720540957],
 [21.066670000000045, 1

In [19]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coordinates = pd.DataFrame(coordinates, columns=['Latitude', 'Longitude'])

In [20]:
# merge the coordinates into the original dataframe
df['Latitude'] =  df_coordinates['Latitude']
df['Longitude'] = df_coordinates['Longitude']
df

,Neighborhood,Latitude,Longitude
0,Ba Đình,21.022010,105.819340
1,Ba Vì,21.083330,105.383330
2,Bắc Từ Liêm,21.062170,105.769410
3,Cầu Giấy,21.035611,105.805973
4,Chương Mỹ,21.039550,105.847930
5,Đan Phượng,21.089790,105.664020
6,Đông Anh,21.155300,105.848720
7,Đống Đa,20.686055,105.736879
8,Gia Lâm,21.016443,105.920923
9,Hà Đông,20.970310,105.781810


## 4. Create a map of Hanoi

In [21]:
# get the coordinates of Hanoi
address = 'Hanoi, Vietnam'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Hanoi, Vietnam {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Hanoi, Vietnam 21.0294498, 105.8544441.


In [22]:
# create map 
map_hanoi = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_hanoi)  
    
map_hanoi

### 5. Define Foursquare Credentials and Version

In [23]:
CLIENT_ID = 'JOMSZEEWVBJYPAKOW2M3AXHEKEBU1WJKE0PVYY4N0CUFZKN3' # Foursquare ID
CLIENT_SECRET = 'CVBOQXI5JJCOVGGCIY4OCEKC54UVIUZGVIZKR1MFO0I4E5SZ' # Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JOMSZEEWVBJYPAKOW2M3AXHEKEBU1WJKE0PVYY4N0CUFZKN3
CLIENT_SECRET:CVBOQXI5JJCOVGGCIY4OCEKC54UVIUZGVIZKR1MFO0I4E5SZ


### 6. Explore neighborhood in dataframe

In [24]:
neighborhood_latitude = df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Ba Đình  are 21.022010000000023, 105.81934000000007.


In [25]:
# get the top 100 venues that are within a radius of 500 meters
radius = 500
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [26]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'NeighborLatitude', 'NeighborLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(316, 7)


,Neighborhood,NeighborLatitude,NeighborLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Ba Đình,21.02201,105.81934,博多幸龍,21.020768,105.817985,Ramen Restaurant
1,Ba Đình,21.02201,105.81934,Newsphinx Beer Bar 2 Láng Hạ,21.023244,105.819343,Restaurant
2,Ba Đình,21.02201,105.81934,Nhật Cường Mobile 12 Láng Hạ,21.020113,105.817417,Tiki Bar
3,Ba Đình,21.02201,105.81934,Café Nhân 1946,21.020023,105.817388,Coffee Shop
4,Ba Đình,21.02201,105.81934,1946,21.018830,105.821899,Vietnamese Restaurant


In [27]:
# Let's check how many venues returned for each Neighborhood
venues_df.groupby(["Neighborhood"]).count()

,NeighborLatitude,NeighborLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Ba Đình,16,16,16,16,16,16
Bắc Từ Liêm,3,3,3,3,3,3
Chương Mỹ,16,16,16,16,16,16
Cầu Giấy,16,16,16,16,16,16
Hai Bà Trưng,1,1,1,1,1,1
Hoàn Kiếm,8,8,8,8,8,8
"Hoàng Mai , Hanoi",21,21,21,21,21,21
Hà Đông,6,6,6,6,6,6
Long Biên,7,7,7,7,7,7


In [28]:
# Find how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

venues_df['VenueCategory'].unique()[:50]

There are 83 uniques categories.


array(['Ramen Restaurant', 'Restaurant', 'Tiki Bar', 'Coffee Shop',
       'Vietnamese Restaurant', 'Asian Restaurant', 'Bubble Tea Shop',
       'Convention Center', 'Karaoke Bar', 'Australian Restaurant',
       'Lake', 'Hotel', 'Dim Sum Restaurant', 'Chinese Restaurant',
       'Seafood Restaurant', 'Gastropub', 'Paintball Field',
       'Japanese Restaurant', 'Beer Bar', 'Cocktail Bar',
       'Korean Restaurant', 'Café', 'Music Venue', 'Sake Bar', 'Gym',
       'BBQ Joint', 'Wings Joint', 'Sandwich Place', 'Dessert Shop',
       'Spa', 'Food', 'Hostel', 'Candy Store', 'Movie Theater', 'Market',
       'Train Station', 'Noodle House', 'Fast Food Restaurant', 'Temple',
       'Thai Restaurant', 'Dry Cleaner', 'Playground', 'Confucian Temple',
       'Fried Chicken Joint', 'Steakhouse', 'Wedding Hall', 'Rock Club',
       'Himalayan Restaurant', 'Mongolian Restaurant', 'Soccer Stadium'],
      dtype=object)

### 7. Analyze each neighborhood

In [29]:
# one hot encoding
hanoi_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
hanoi_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = list(hanoi_onehot.columns[-1:]) + list(hanoi_onehot.columns[:-1])
hanoi_onehot = hanoi_onehot[fixed_columns]

print(hanoi_onehot.shape)
hanoi_onehot.head()

(316, 84)


,Neighborhoods,Arepa Restaurant,Arts & Crafts Store,Asian Restaurant,Australian Restaurant,BBQ Joint,Bakery,Bar,Beer Bar,Beer Garden,Bookstore,Bubble Tea Shop,Burger Joint,Cafeteria,Café,Candy Store,Chinese Restaurant,Cocktail Bar,Coffee Shop,Confucian Temple,Convention Center,Dessert Shop,Dim Sum Restaurant,Dry Cleaner,Farm,Fast Food Restaurant,Food,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Gastropub,Gym,Halal Restaurant,Himalayan Restaurant,History Museum,Hostel,Hotel,Hotpot Restaurant,Ice Cream Shop,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Lounge,Market,Massage Studio,Men's Store,Mobile Phone Shop,Mongolian Restaurant,Movie Theater,Multiplex,Music Venue,Noodle House,North Indian Restaurant,Paintball Field,Pastry Shop,Pizza Place,Playground,Pub,Ramen Restaurant,Restaurant,Rock Club,Sake Bar,Salad Place,Sandwich Place,Seafood Restaurant,Shopping Mall,Soccer Stadium,Spa,Steakhouse,Sushi Restaurant,Tea Room,Temple,Thai Restaurant,Theater,Tiki Bar,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wedding Hall,Wings Joint
0,Ba Đình,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Ba Đình,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Ba Đình,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,Ba Đình,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Ba Đình,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [30]:
# let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

hanoi_grouped = hanoi_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(hanoi_grouped.shape)
hanoi_grouped

(18, 84)


,Neighborhoods,Arepa Restaurant,Arts & Crafts Store,Asian Restaurant,Australian Restaurant,BBQ Joint,Bakery,Bar,Beer Bar,Beer Garden,Bookstore,Bubble Tea Shop,Burger Joint,Cafeteria,Café,Candy Store,Chinese Restaurant,Cocktail Bar,Coffee Shop,Confucian Temple,Convention Center,Dessert Shop,Dim Sum Restaurant,Dry Cleaner,Farm,Fast Food Restaurant,Food,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Gastropub,Gym,Halal Restaurant,Himalayan Restaurant,History Museum,Hostel,Hotel,Hotpot Restaurant,Ice Cream Shop,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Lounge,Market,Massage Studio,Men's Store,Mobile Phone Shop,Mongolian Restaurant,Movie Theater,Multiplex,Music Venue,Noodle House,North Indian Restaurant,Paintball Field,Pastry Shop,Pizza Place,Playground,Pub,Ramen Restaurant,Restaurant,Rock Club,Sake Bar,Salad Place,Sandwich Place,Seafood Restaurant,Shopping Mall,Soccer Stadium,Spa,Steakhouse,Sushi Restaurant,Tea Room,Temple,Thai Restaurant,Theater,Tiki Bar,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wedding Hall,Wings Joint
0,Ba Đình,0.000000,0.00,0.062500,0.0625,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0625,0.00,0.00,0.000000,0.0000,0.0625,0.0000,0.062500,0.000000,0.0625,0.000000,0.0625,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000,0.000000,0.0000,0.125000,0.000000,0.00,0.000000,0.000000,0.00,0.0625,0.000000,0.062500,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.0625,0.062500,0.000000,0.0000,0.000000,0.000000,0.062500,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0625,0.0,0.000000,0.062500,0.000000,0.000000
1,Bắc Từ Liêm,0.000000,0.00,0.000000,0.0000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0000,0.00,0.00,0.000000,0.0000,0.0000,0.0000,0.000000,0.000000,0.0000,0.000000,0.0000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.0000,0.00,0.000000,0.000000,0.0000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000,0.00,0.0000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.333333,0.000000,0.000000
2,Chương Mỹ,0.000000,0.00,0.000000,0.0000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0000,0.00,0.00,0.062500,0.0625,0.0000,0.0625,0.187500,0.000000,0.0000,0.125000,0.0000,0.000,0.000000,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000,0.000000,0.0625,0.125000,0.000000,0.00,0.000000,0.000000,0.00,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.0000,0.000000,0.000000,0.0000,0.000000,0.062500,0.062500,0.000000,0.000000,0.0625,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.062500,0.000000,0.000000
3,Cầu Giấy,0.000000,0.00,0.000000,0.0000,0.062500,0.000000,0.000000,0.0625,0.000000,0.000000,0.0625,0.00,0.00,0.062500,0.0000,0.0000,0.0625,0.062500,0.000000,0.0000,0.000000,0.0000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0625,0.00,0.000000,0.000000,0.0000,0.000000,0.000000,0.00,0.000000,0.125000,0.00,0.0000,0.062500,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.0000,0.000000,0.000000,0.0625,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.187500,0.000000,0.062500
4,Hai Bà Trưng,0.000000,0.00,0.000000,0.0000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0000,0.00,0.00,0.000000,0.0000,0.0000,0.0000,0.000000,0.000000,0.0000,0.00

**Now let's create the new dataframe and display the top 10 venues for each Neighborhood**

In [31]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhoods'] = hanoi_grouped['Neighborhoods']

for ind in np.arange(hanoi_grouped.shape[0]):
    row_categories = hanoi_grouped.iloc[ind, :].iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 1:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

(18, 11)


,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ba Đình,Hotel,Lake,Convention Center,Coffee Shop,Ramen Restaurant,Restaurant,Chinese Restaurant,Seafood Restaurant,Bubble Tea Shop,Dim Sum Restaurant
1,Bắc Từ Liêm,Gastropub,Vietnamese Restaurant,Paintball Field,Frozen Yogurt Shop,Dry Cleaner,Farm,Fast Food Restaurant,Food,Food Truck,French Restaurant
2,Chương Mỹ,Coffee Shop,Dessert Shop,Hotel,Cocktail Bar,Candy Store,Café,Food,Sandwich Place,Seafood Restaurant,Spa
3,Cầu Giấy,Vietnamese Restaurant,Japanese Restaurant,Wings Joint,Bubble Tea Shop,Music Venue,Coffee Shop,Cocktail Bar,Sake Bar,Café,Gym
4,Hai Bà Trưng,Temple,Wings Joint,Gastropub,Dry Cleaner,Farm,Fast Food Restaurant,Food,Food Truck,French Restaurant,Fried Chicken Joint
5,Hoàn Kiếm,Vietnamese Restaurant,Asian Restaurant,Dry Cleaner,Thai Restaurant,Playground,Wings Joint,Fried Chicken Joint,Dim Sum Restaurant,Farm,Fast Food Restaurant
6,"Hoàng Mai , Hanoi",Wings Joint,Rock Club,Hotel,Japanese Restaurant,Wedding Hall,Fried Chicken Joint,Lounge,Mongolian Restaurant,Dessert Shop,Noodle House
7,Hà Đông,Dessert Shop,Vietnamese Restaurant,Fast Food Restaurant,Noodle House,Market,Café,Fried Chicken Joint,Dry Cleaner,Farm,Food
8,Long Biên,Market,Seafood Restaurant,Vietnamese Restaurant,Asian Restaurant,BBQ Joint,Tea Room,Pastry Shop,Dry Cleaner,Farm,Fast Food Restaurant
9,Mê Linh,Temple,Wings Joint,Gastropub,Dry Cleaner,Farm,Fast Food Restaurant,Food,Food Truck,French Restaurant,Fried Chicken Joint


### 8. Cluster neighborhood

**Run k-means to cluster Hanoi into 4 clusters**

In [39]:
# set number of clusters
kclusters = 4

hanoi_grouped_clustering = hanoi_grouped.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(hanoi_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 2, 2, 1, 2, 2, 2, 2, 1], dtype=int32)

In [40]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
hanoi_merged = hanoi_grouped.copy()

# add clustering labels
hanoi_merged["Cluster Labels"] = kmeans.labels_

In [41]:
hanoi_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
hanoi_merged.head()

,Neighborhood,Arepa Restaurant,Arts & Crafts Store,Asian Restaurant,Australian Restaurant,BBQ Joint,Bakery,Bar,Beer Bar,Beer Garden,Bookstore,Bubble Tea Shop,Burger Joint,Cafeteria,Café,Candy Store,Chinese Restaurant,Cocktail Bar,Coffee Shop,Confucian Temple,Convention Center,Dessert Shop,Dim Sum Restaurant,Dry Cleaner,Farm,Fast Food Restaurant,Food,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Gastropub,Gym,Halal Restaurant,Himalayan Restaurant,History Museum,Hostel,Hotel,Hotpot Restaurant,Ice Cream Shop,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Lounge,Market,Massage Studio,Men's Store,Mobile Phone Shop,Mongolian Restaurant,Movie Theater,Multiplex,Music Venue,Noodle House,North Indian Restaurant,Paintball Field,Pastry Shop,Pizza Place,Playground,Pub,Ramen Restaurant,Restaurant,Rock Club,Sake Bar,Salad Place,Sandwich Place,Seafood Restaurant,Shopping Mall,Soccer Stadium,Spa,Steakhouse,Sushi Restaurant,Tea Room,Temple,Thai Restaurant,Theater,Tiki Bar,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wedding Hall,Wings Joint,Cluster Labels
0,Ba Đình,0.0,0.0,0.0625,0.0625,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0625,0.0,0.0,0.0000,0.0000,0.0625,0.0000,0.0625,0.0,0.0625,0.000,0.0625,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.000000,0.0000,0.0,0.0,0.0,0.0000,0.125,0.0,0.0,0.0,0.000,0.0,0.0625,0.0000,0.0625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0625,0.0625,0.0,0.0000,0.0,0.0000,0.0625,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0625,0.0,0.0,0.062500,0.0,0.0000,2
1,Bắc Từ Liêm,0.0,0.0,0.0000,0.0000,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0000,0.000,0.0000,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.333333,0.0000,0.0,0.0,0.0,0.0000,0.000,0.0,0.0,0.0,0.000,0.0,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0000,0.0,0.0000,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.333333,0.0,0.0000,2
2,Chương Mỹ,0.0,0.0,0.0000,0.0000,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0625,0.0625,0.0000,0.0625,0.1875,0.0,0.0000,0.125,0.0000,0.0,0.0,0.0,0.0625,0.0,0.0,0.0,0.0,0.000000,0.0000,0.0,0.0,0.0,0.0625,0.125,0.0,0.0,0.0,0.000,0.0,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0000,0.0,0.0625,0.0625,0.0,0.0,0.0625,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.062500,0.0,0.0000,2
3,Cầu Giấy,0.0,0.0,0.0000,0.0000,0.0625,0.0,0.0,0.0625,0.0,0.0,0.0625,0.0,0.0,0.0625,0.0000,0.0000,0.0625,0.0625,0.0,0.0000,0.000,0.0000,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.000000,0.0625,0.0,0.0,0.0,0.0000,0.000,0.0,0.0,0.0,0.125,0.0,0.0000,0.0625,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0625,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0625,0.0,0.0000,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.187500,0.0,0.0625,2
4,Hai Bà Trưng,0.0,0.0,0.0000,0.0000,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0000,0.000,0.0000,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.000000,0.0000,0.0,0.0,0.0,0.0000,0.000,0.0,0.0,0.0,0.000,0.0,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0000,0.0,0.0000,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0,1.0,0.0,0.0,0.0000,0.0,0.0,0.000000,0.0,0.0000,1


In [42]:
# merge hanoi_grouped with df to add latitude/longitude for each neighborhood
hanoi_merged = hanoi_merged.join(df.set_index("Neighborhood"), on="Neighborhood")

print(hanoi_merged.shape)
hanoi_merged.head() # check the last columns!

(18, 87)


,Neighborhood,Arepa Restaurant,Arts & Crafts Store,Asian Restaurant,Australian Restaurant,BBQ Joint,Bakery,Bar,Beer Bar,Beer Garden,Bookstore,Bubble Tea Shop,Burger Joint,Cafeteria,Café,Candy Store,Chinese Restaurant,Cocktail Bar,Coffee Shop,Confucian Temple,Convention Center,Dessert Shop,Dim Sum Restaurant,Dry Cleaner,Farm,Fast Food Restaurant,Food,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Gastropub,Gym,Halal Restaurant,Himalayan Restaurant,History Museum,Hostel,Hotel,Hotpot Restaurant,Ice Cream Shop,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Lounge,Market,Massage Studio,Men's Store,Mobile Phone Shop,Mongolian Restaurant,Movie Theater,Multiplex,Music Venue,Noodle House,North Indian Restaurant,Paintball Field,Pastry Shop,Pizza Place,Playground,Pub,Ramen Restaurant,Restaurant,Rock Club,Sake Bar,Salad Place,Sandwich Place,Seafood Restaurant,Shopping Mall,Soccer Stadium,Spa,Steakhouse,Sushi Restaurant,Tea Room,Temple,Thai Restaurant,Theater,Tiki Bar,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wedding Hall,Wings Joint,Cluster Labels,Latitude,Longitude
0,Ba Đình,0.0,0.0,0.0625,0.0625,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0625,0.0,0.0,0.0000,0.0000,0.0625,0.0000,0.0625,0.0,0.0625,0.000,0.0625,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.000000,0.0000,0.0,0.0,0.0,0.0000,0.125,0.0,0.0,0.0,0.000,0.0,0.0625,0.0000,0.0625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0625,0.0625,0.0,0.0000,0.0,0.0000,0.0625,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0625,0.0,0.0,0.062500,0.0,0.0000,2,21.022010,105.819340
1,Bắc Từ Liêm,0.0,0.0,0.0000,0.0000,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0000,0.000,0.0000,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.333333,0.0000,0.0,0.0,0.0,0.0000,0.000,0.0,0.0,0.0,0.000,0.0,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0000,0.0,0.0000,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.333333,0.0,0.0000,2,21.062170,105.769410
2,Chương Mỹ,0.0,0.0,0.0000,0.0000,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0625,0.0625,0.0000,0.0625,0.1875,0.0,0.0000,0.125,0.0000,0.0,0.0,0.0,0.0625,0.0,0.0,0.0,0.0,0.000000,0.0000,0.0,0.0,0.0,0.0625,0.125,0.0,0.0,0.0,0.000,0.0,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0000,0.0,0.0625,0.0625,0.0,0.0,0.0625,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.062500,0.0,0.0000,2,21.039550,105.847930
3,Cầu Giấy,0.0,0.0,0.0000,0.0000,0.0625,0.0,0.0,0.0625,0.0,0.0,0.0625,0.0,0.0,0.0625,0.0000,0.0000,0.0625,0.0625,0.0,0.0000,0.000,0.0000,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.000000,0.0625,0.0,0.0,0.0,0.0000,0.000,0.0,0.0,0.0,0.125,0.0,0.0000,0.0625,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0625,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0625,0.0,0.0000,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.187500,0.0,0.0625,2,21.035611,105.805973
4,Hai Bà Trưng,0.0,0.0,0.0000,0.0000,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0000,0.000,0.0000,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.000000,0.0000,0.0,0.0,0.0,0.0000,0.000,0.0,0.0,0.0,0.000,0.0,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0000,0.0,0.0000,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0,1.0,0.0,0.0,0.0000,0.0,0.0,0.000000,0.0,0.0000,1,21.155050,105.734290


In [43]:
# sort the results by Cluster Labels
print(hanoi_merged.shape)
hanoi_merged.sort_values(["Cluster Labels"], inplace=True)
hanoi_merged

(18, 87)


,Neighborhood,Arepa Restaurant,Arts & Crafts Store,Asian Restaurant,Australian Restaurant,BBQ Joint,Bakery,Bar,Beer Bar,Beer Garden,Bookstore,Bubble Tea Shop,Burger Joint,Cafeteria,Café,Candy Store,Chinese Restaurant,Cocktail Bar,Coffee Shop,Confucian Temple,Convention Center,Dessert Shop,Dim Sum Restaurant,Dry Cleaner,Farm,Fast Food Restaurant,Food,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Gastropub,Gym,Halal Restaurant,Himalayan Restaurant,History Museum,Hostel,Hotel,Hotpot Restaurant,Ice Cream Shop,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Lounge,Market,Massage Studio,Men's Store,Mobile Phone Shop,Mongolian Restaurant,Movie Theater,Multiplex,Music Venue,Noodle House,North Indian Restaurant,Paintball Field,Pastry Shop,Pizza Place,Playground,Pub,Ramen Restaurant,Restaurant,Rock Club,Sake Bar,Salad Place,Sandwich Place,Seafood Restaurant,Shopping Mall,Soccer Stadium,Spa,Steakhouse,Sushi Restaurant,Tea Room,Temple,Thai Restaurant,Theater,Tiki Bar,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wedding Hall,Wings Joint,Cluster Labels,Latitude,Longitude
16,Đan Phượng,0.000000,0.00,0.000000,0.0000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0000,0.00,0.00,0.000000,0.0000,0.0000,0.0000,0.000000,0.000000,0.0000,0.000000,0.0000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000,0.000000,0.0000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.0000,0.000000,0.000000,0.000000,0.500000,0.00,0.000000,0.00,0.000000,0.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.0000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0,21.089790,105.664020
4,Hai Bà Trưng,0.000000,0.00,0.000000,0.0000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0000,0.00,0.00,0.000000,0.0000,0.0000,0.0000,0.000000,0.000000,0.0000,0.000000,0.0000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000,0.000000,0.0000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.0000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,1,21.155050,105.734290
9,Mê Linh,0.000000,0.00,0.000000,0.0000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0000,0.00,0.00,0.000000,0.0000,0.0000,0.0000,0.000000,0.000000,0.0000,0.000000,0.0000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000,0.000000,0.0000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.0000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,1,21.159590,105.734800
0,Ba Đình,0.000000,0.00,0.062500,0.0625,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0625,0.00,0.00,0.000000,0.0000,0.0625,0.0000,0.062500,0.000000,0.0625,0.000000,0.0625,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000,0.000000,0.0000,0.125000,0.000000,0.00,0.000000,0.000000,0.00,0.0625,0.000000,0.062500,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.0625,0.062500,0.000000,0.0000,0.000000,0.000000,0.062500,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0625,0.0,0.000000,0.062500,0.000000,0.000000,2,21.022010,105.819340
15,Tây Hồ,0.000000,0.00,0.000000,0.0000,0.0000

**Let's visualize**

In [44]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(hanoi_merged['Latitude'], hanoi_merged['Longitude'], hanoi_merged['Neighborhood'], hanoi_merged['Cluster Labels']):
    label = folium.Popup('{} - Cluster {}'.format(poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 12. Examine the clusters

**Cluster 1**

In [50]:
hanoi_merged.loc[hanoi_merged['Cluster Labels'] == 0, hanoi_merged.columns[[0] + list(range(5, hanoi_merged.shape[1]))]]

,Neighborhood,BBQ Joint,Bakery,Bar,Beer Bar,Beer Garden,Bookstore,Bubble Tea Shop,Burger Joint,Cafeteria,Café,Candy Store,Chinese Restaurant,Cocktail Bar,Coffee Shop,Confucian Temple,Convention Center,Dessert Shop,Dim Sum Restaurant,Dry Cleaner,Farm,Fast Food Restaurant,Food,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Gastropub,Gym,Halal Restaurant,Himalayan Restaurant,History Museum,Hostel,Hotel,Hotpot Restaurant,Ice Cream Shop,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Lounge,Market,Massage Studio,Men's Store,Mobile Phone Shop,Mongolian Restaurant,Movie Theater,Multiplex,Music Venue,Noodle House,North Indian Restaurant,Paintball Field,Pastry Shop,Pizza Place,Playground,Pub,Ramen Restaurant,Restaurant,Rock Club,Sake Bar,Salad Place,Sandwich Place,Seafood Restaurant,Shopping Mall,Soccer Stadium,Spa,Steakhouse,Sushi Restaurant,Tea Room,Temple,Thai Restaurant,Theater,Tiki Bar,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wedding Hall,Wings Joint,Cluster Labels,Latitude,Longitude
16,Đan Phượng,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,21.08979,105.66402


**Cluster 2**

In [51]:
hanoi_merged.loc[hanoi_merged['Cluster Labels'] == 1, hanoi_merged.columns[[0] + list(range(5, hanoi_merged.shape[1]))]]

,Neighborhood,BBQ Joint,Bakery,Bar,Beer Bar,Beer Garden,Bookstore,Bubble Tea Shop,Burger Joint,Cafeteria,Café,Candy Store,Chinese Restaurant,Cocktail Bar,Coffee Shop,Confucian Temple,Convention Center,Dessert Shop,Dim Sum Restaurant,Dry Cleaner,Farm,Fast Food Restaurant,Food,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Gastropub,Gym,Halal Restaurant,Himalayan Restaurant,History Museum,Hostel,Hotel,Hotpot Restaurant,Ice Cream Shop,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Lounge,Market,Massage Studio,Men's Store,Mobile Phone Shop,Mongolian Restaurant,Movie Theater,Multiplex,Music Venue,Noodle House,North Indian Restaurant,Paintball Field,Pastry Shop,Pizza Place,Playground,Pub,Ramen Restaurant,Restaurant,Rock Club,Sake Bar,Salad Place,Sandwich Place,Seafood Restaurant,Shopping Mall,Soccer Stadium,Spa,Steakhouse,Sushi Restaurant,Tea Room,Temple,Thai Restaurant,Theater,Tiki Bar,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wedding Hall,Wings Joint,Cluster Labels,Latitude,Longitude
4,Hai Bà Trưng,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,21.15505,105.73429
9,Mê Linh,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,21.15959,105.73480


**Cluster 3**

In [52]:
hanoi_merged.loc[hanoi_merged['Cluster Labels'] == 2, hanoi_merged.columns[[0] + list(range(5, hanoi_merged.shape[1]))]]

,Neighborhood,BBQ Joint,Bakery,Bar,Beer Bar,Beer Garden,Bookstore,Bubble Tea Shop,Burger Joint,Cafeteria,Café,Candy Store,Chinese Restaurant,Cocktail Bar,Coffee Shop,Confucian Temple,Convention Center,Dessert Shop,Dim Sum Restaurant,Dry Cleaner,Farm,Fast Food Restaurant,Food,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Gastropub,Gym,Halal Restaurant,Himalayan Restaurant,History Museum,Hostel,Hotel,Hotpot Restaurant,Ice Cream Shop,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Lounge,Market,Massage Studio,Men's Store,Mobile Phone Shop,Mongolian Restaurant,Movie Theater,Multiplex,Music Venue,Noodle House,North Indian Restaurant,Paintball Field,Pastry Shop,Pizza Place,Playground,Pub,Ramen Restaurant,Restaurant,Rock Club,Sake Bar,Salad Place,Sandwich Place,Seafood Restaurant,Shopping Mall,Soccer Stadium,Spa,Steakhouse,Sushi Restaurant,Tea Room,Temple,Thai Restaurant,Theater,Tiki Bar,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wedding Hall,Wings Joint,Cluster Labels,Latitude,Longitude
0,Ba Đình,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0625,0.00,0.00,0.000000,0.0000,0.0625,0.0000,0.062500,0.000000,0.0625,0.000000,0.0625,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000,0.000000,0.0000,0.125000,0.000000,0.00,0.000000,0.000000,0.00,0.0625,0.000000,0.062500,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.0625,0.062500,0.000000,0.0000,0.000000,0.000000,0.062500,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0625,0.0,0.000000,0.062500,0.000000,0.000000,2,21.022010,105.819340
15,Tây Hồ,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0000,0.00,0.00,0.000000,0.0000,0.0000,0.0000,0.000000,0.000000,0.0000,0.000000,0.0000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000,0.000000,0.0000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.0000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.250000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.0000,0.0,0.000000,0.500000,0.000000,0.000000,2,21.066670,105.833330
14,Thanh Xuân,0.166667,0.166667,0.000000,0.0000,0.000000,0.166667,0.0000,0.00,0.00,0.000000,0.0000,0.0000,0.0000,0.000000,0.000000,0.0000,0.000000,0.0000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000,0.000000,0.0000,0.166667,0.000000,0.00,0.000000,0.000000,0.00,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.0000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.166667,0.000000,0.000000,2,21.037600,105.775070
13,"Sơn Tây, Hanoi",0.037037,0.037037,0.037037,0.0000,0.000000,0.037037,0.0000,0.00,0.00,0.074074,0.0000,0.0000,0.0000,0.074074,0.000000,0.0000,0.000000,0.0000,0.000,0.000000,0.000000,0.037037,0.000000,0.000000,0.037037,0.000000,0.000000,0.0000,0.00,0.000000,0.037037,0.0000,0.037037,0.037037,0.00,0.037037,0.074074,0.00,0.0000,0.000000,0.037037,0.037037,0.037037,0.00,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.0000,0.037037,0.037037,0.0000,0.000000,0.000000,0.000000,0.000000,0.037037,0.0000,0.000000,0.000000,0.000000,0.037037,0.000000,0.000000,0.0000,0.0,0.037037,0.037037,0.037037,0.037037,2,21.032796,105.830137
12,Phúc Thọ,0.000000,0.010000,0.010000,0.0000,0.010000,0.000000,0.0000,0.01,0.01,0.060000,0.0000,0.0000,0.0100,0.140000,0.000000,0.0000,0.020000,0.0000,0.000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.010000,0.0000,

**Cluster 4**

In [53]:
hanoi_merged.loc[hanoi_merged['Cluster Labels'] == 3, hanoi_merged.columns[[0] + list(range(5, hanoi_merged.shape[1]))]]

,Neighborhood,BBQ Joint,Bakery,Bar,Beer Bar,Beer Garden,Bookstore,Bubble Tea Shop,Burger Joint,Cafeteria,Café,Candy Store,Chinese Restaurant,Cocktail Bar,Coffee Shop,Confucian Temple,Convention Center,Dessert Shop,Dim Sum Restaurant,Dry Cleaner,Farm,Fast Food Restaurant,Food,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Gastropub,Gym,Halal Restaurant,Himalayan Restaurant,History Museum,Hostel,Hotel,Hotpot Restaurant,Ice Cream Shop,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Lounge,Market,Massage Studio,Men's Store,Mobile Phone Shop,Mongolian Restaurant,Movie Theater,Multiplex,Music Venue,Noodle House,North Indian Restaurant,Paintball Field,Pastry Shop,Pizza Place,Playground,Pub,Ramen Restaurant,Restaurant,Rock Club,Sake Bar,Salad Place,Sandwich Place,Seafood Restaurant,Shopping Mall,Soccer Stadium,Spa,Steakhouse,Sushi Restaurant,Tea Room,Temple,Thai Restaurant,Theater,Tiki Bar,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wedding Hall,Wings Joint,Cluster Labels,Latitude,Longitude
17,Đông Anh,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3,21.1553,105.84872
